In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib;


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
!pip install openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
!pip install pytz
!pip install tzlocal


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


## GPT Function - get_schedule

In [5]:
get_schedule_schema = {
  "name": "get_schedule",
  "description": "Get my schedule from calendar for certain time period",
  "parameters": {
    "type": "object",
    "properties": {
      "period": {
        "type": "string",
        "enum": [
          "today",
          "week"
        ]
      }
    },
    "required": [
      "period"
    ]
  }
}

In [6]:
from Calendar import get_events, get_time_range

def simplify_event(event):
    return {
        'summary': event['summary'],
        'start': event['start']['dateTime'],
        'end': event['end']['dateTime']
    }

## define function for Assistant
def get_schedule(params): # {period: string} period is an enum: today|week
    events = get_events(params['period'])
    simplified_events = [simplify_event(event) for event in events]
    return {
        'schedules': simplified_events
    }

data/client_secret_calendar.json-calendar-v3-(['https://www.googleapis.com/auth/calendar'],)
['https://www.googleapis.com/auth/calendar']
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=71060035146-lvo9kdt1t9icvcl3h33t7j039e2t9hve.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=cMLWCCEzuQTH6EPfEM3cPjqEsDjvfV&access_type=offline
calendar service created successfully
Calendar: lvjian700@gmail.com
Timezone: Australia/Melbourne


## GPT Function - book_evnet

In [7]:
book_event_schema = {
  "name": "book_event",
  "parameters": {
    "type": "object",
    "properties": {
      "title": {
        "type": "string",
        "description": "A summary of the event"
      },
      "startTime": {
        "type": "string",
        "description": "A start time the event, Converting it to iso format"
      },
      "durationInMins": {
        "type": "integer",
        "description": "duration in minute for the event"
      }
    },
    "required": [
      "title"
    ]
  },
  "description": "Book an event on calendar."
}

In [8]:
from datetime import datetime, timedelta
from Calendar import insert_event, time_to_iso_str
from dateutil.parser import isoparse

def book_event(params):
    print(f"Booking an event", params)
    title = params['title']
    start_datetime = isoparse(params['startTime'])
    end_datetime = start_datetime + timedelta(minutes=params['durationInMins'])

    print(f"Book event at: {start_datetime} ~ {end_datetime}")

    event_body = {
        'summary': title,
        'start': {
            'dateTime': start_datetime.isoformat()
        },
        'end': {
            'dateTime': end_datetime.isoformat()
        }
    }
    print(event_body)
    event = insert_event(event_body)
    #print(event)
    return simplify_event(event)

In [9]:
# book_event({
#     'title': 'Take a break',
#     'startTime': '2024-02-09T10:30:00+11:00',
#     'durationInMins': 15
# })

## Assistants

In [10]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [11]:
from openai import OpenAI
from Calendar import current_timezone

client = OpenAI()

def create_assistant(name, instructions, model="gpt-3.5-turbo-1106"):
    return client.beta.assistants.create(
        name=name,
        instructions=instructions,
        model=model,
    )

assistant = create_assistant(
    name = "Working Assistant",
    instructions = f"""You are a helpful work assistant,\
    assistant me for daily working task like checking or booking calendar.\
    Using Timezone: {current_timezone}
    """
)
show_json(assistant)

{'id': 'asst_7pFWp3MOsgggaFRcS4zh9NMk',
 'created_at': 1707788970,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a helpful work assistant,    assistant me for daily working task like checking or booking calendar.    Using Timezone: Australia/Melbourne\n    ',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Working Assistant',
 'object': 'assistant',
 'tools': []}

### Assistant Utils

In [12]:
from Calendar import tz_now
from Calendar import current_timezone
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )

    additional_instructions = f"""
        now: {tz_now().isoformat()} \
        timezone: {current_timezone}"""

    print(f"submit_message with additional_instructions: {additional_instructions}")
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
        additional_instructions=additional_instructions
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(assistant.id, thread, user_input)
    return thread, run

def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

### Assist Calendar - Setup

In [13]:
assistant = client.beta.assistants.update(
    assistant.id,
    #model="gpt-4-1106-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "function", "function": get_schedule_schema},
        {"type": "function", "function": book_event_schema}
    ],
)
show_json(assistant)

{'id': 'asst_7pFWp3MOsgggaFRcS4zh9NMk',
 'created_at': 1707788970,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a helpful work assistant,    assistant me for daily working task like checking or booking calendar.    Using Timezone: Australia/Melbourne\n    ',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Working Assistant',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'function': {'name': 'get_schedule',
    'description': 'Get my schedule from calendar for certain time period',
    'parameters': {'type': 'object',
     'properties': {'period': {'type': 'string', 'enum': ['today', 'week']}},
     'required': ['period']}},
   'type': 'function'},
  {'function': {'name': 'book_event',
    'description': 'Book an event on calendar.',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string',
       'description': 'A summary of the event'},
      'startTime': {'type': 'string',
       'description': 'A start t

In [14]:
def execute_func(func_name, arguments):
    output = None

    if func_name == 'get_schedule':
        output = get_schedule(arguments)
    elif func_name == 'book_event':
        output = book_event(arguments)

    return output

def prompt(message, thread):
    run = submit_message(assistant.id, thread, message)
    run = wait_on_run(run, thread)

    if run.status == 'requires_action':
        # Extract single tool call,
        tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        output = None
        try:
            output = execute_func(func_name, arguments)
        except Exception as e:
            print(f"An error occurred while executing {func_name}: {str(e)}")
            client.beta.threads.runs.cancel(run_id=run.id, thread_id=thread.id)

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=[
                {
                    "tool_call_id": tool_call.id,
                    "output": json.dumps(output),
                }
            ],
        )
        run = wait_on_run(run, thread)

    pretty_print(get_response(thread))



In [15]:
from functools import partial

def start_conversation():
    thread = client.beta.threads.create()
    return partial(prompt, thread = thread)

## Usage

In [20]:
chat = start_conversation()

In [17]:
chat("What's my schedule today")

submit_message with additional_instructions: 
        now: 2024-02-13T12:49:31.709329+11:00         timezone: Australia/Melbourne
# Messages
user: What's my schedule today
assistant: It looks like you don't have any scheduled events for today. If you'd like to schedule an event, please let me know, and I can assist with that!



In [18]:
chat("Reminder me to cancel leetcode subscription at 1 Feb 2025")

submit_message with additional_instructions: 
        now: 2024-02-13T12:49:38.747698+11:00         timezone: Australia/Melbourne
Booking an event {'title': 'Cancel LeetCode Subscription', 'startTime': '2025-02-01T09:00:00', 'durationInMins': 30}
Book event at: 2025-02-01 09:00:00 ~ 2025-02-01 09:30:00
{'summary': 'Cancel LeetCode Subscription', 'start': {'dateTime': '2025-02-01T09:00:00'}, 'end': {'dateTime': '2025-02-01T09:30:00'}}
An error occurred while executing book_event: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/lvjian700%40gmail.com/events?alt=json returned "Missing time zone definition for start time.". Details: "[{'domain': 'global', 'reason': 'required', 'message': 'Missing time zone definition for start time.'}, {'domain': 'global', 'reason': 'required', 'message': 'Missing time zone definition for end time.'}]">


BadRequestError: Error code: 400 - {'error': {'message': 'Runs in status "cancelled" do not accept tool outputs.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
#chat("When is my last session start? what's the duration?")

In [ ]:
#chat("Book a 15mins streching session before my last session")

In [ ]:
chat("What is the current timezone")

In [ ]:
chat("Book a 30mins catchup with Pavi at 10:30 this THU")

In [ ]:
chat("Take a break until 5:00PM")

In [ ]:
chat("Book a focus time until 5:00PM today") # This works perfect with GPT 4

In [21]:
chat("I'm exhausted of back to back meetings this morning")

submit_message with additional_instructions: 
        now: 2024-02-13T12:51:23.348506+11:00         timezone: Australia/Melbourne
# Messages
user: I'm exhausted of back to back meetings this morning
assistant: I understand how exhausting that can be. Would you like me to check your schedule and see if there's a chance to book a break for you?



In [22]:
chat("Sure")

submit_message with additional_instructions: 
        now: 2024-02-13T12:51:55.199810+11:00         timezone: Australia/Melbourne
# Messages
user: I'm exhausted of back to back meetings this morning
assistant: I understand how exhausting that can be. Would you like me to check your schedule and see if there's a chance to book a break for you?
user: Sure
assistant: It looks like there are no current events scheduled for today. Let me find a suitable time slot for a break.



In [24]:
chat("Book 15mins asap")

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_PqjVdgj8vqs3wwSbJp8QdaRw while a run run_J911Ut1dHqOgf13NyHe2DhRV is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}